# app

> Fill in a module description here

In [ ]:
# | default_exp watcher/app

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import hashlib
import logging
import os
import pathlib
import time

import click
from sqlmodel import Field, Session, SQLModel, create_engine, select
from watchdog.events import LoggingEventHandler
from watchdog.observers import Observer

In [ ]:
# | export

DEFAULT_DB_PATH = str(pathlib.Path.home() / "ram_boost/db.sqlite")

In [ ]:
# | export


class CustomEventHandler(LoggingEventHandler):
    def __init__(self, key=None, delete=False):
        self.key = key
        self.delete = delete

    def dispatch(self, event):
        print("CHANGED: ", event)

        if event.event_type == "created" or event.event_type == "modified":
            pass

        if self.delete and event.event_type == "deleted":
            pass

In [ ]:
# | export


class File(SQLModel, table=True):
    path: str = Field(primary_key=True)
    hash: str

In [ ]:
# | export


class File(SQLModel, table=True):
    path: str = Field(primary_key=True)
    hash: str

In [ ]:
# | export


def hash(path: str):
    "https://stackoverflow.com/a/44873382"
    h = hashlib.sha256()
    b = bytearray(128 * 1024)
    mv = memoryview(b)

    with open(path, "rb", buffering=0) as f:
        while n := f.readinto(mv):
            h.update(mv[:n])

    return h.hexdigest()

In [ ]:
# | export


@click.command()
@click.argument("path", type=click.Path(exists=True))
@click.option("--exclude", "-e", multiple=True, help="Exclude paths")
@click.option("--key", "-k", default=os.environ.get("RB_WATCHER_KEY"), help="API key")
@click.option("--database", "-d", help="SQLite database URL")
@click.option("--delete", "-D", is_flag=True, help="Delete files from the server")
@click.option("--interval", "-i", default=1, help="Interval to check for changes")
@click.option("--verbose", "-v", is_flag=True, help="Verbose output")
def watch(path, exclude, key, database, delete, interval, verbose):
    paths = [
        p
        for p in pathlib.Path(path).rglob("*")
        if p.is_file() and not any(ex in str(p) for ex in exclude)
    ]

    DB_PATH = database or DEFAULT_DB_PATH
    DB_FOLDER = pathlib.Path(DB_PATH).parent
    DB_FOLDER.mkdir(parents=True, exist_ok=True)

    engine = create_engine("sqlite:///" + DB_PATH)
    SQLModel.metadata.create_all(engine)

    with Session(engine) as session:
        statement = select(File)
        files_saved = session.exec(statement).all()
        paths_saved = [f.path for f in files_saved]

        paths_new = [str(p) for p in paths if str(p) not in paths_saved]
        paths_new_hashed = [File(path=str(p), hash=hash(str(p))) for p in paths_new]

        # Send the new files the the server to be chunked and stored

        Session.bulk_insert_mappings(session, File, paths_new_hashed)

        session.commit()

        observer = Observer()
        event_handler = CustomEventHandler(key=key)
        observer.schedule(event_handler, path=path, recursive=True)
        observer.start()

        try:
            while True:
                time.sleep(interval)
        except KeyboardInterrupt:
            observer.stop()
            observer.join()

In [ ]:
# | export
# | eval: false


if __name__ == "__main__":
    watch()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()